In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df = pd.read_csv('WELLCOME/WELLCOME_APCspend2013_forThinkful.csv', encoding='ansi')

print(df.columns)
print(df.head())

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')
              PMID/PMCID Publisher           Journal title  \
0                    NaN       CUP  Psychological Medicine   
1             PMC3679557       ACS       Biomacromolecules   
2  23043264  PMC3506128        ACS              J Med Chem   
3    23438330 PMC3646402       ACS              J Med Chem   
4   23438216 PMC3601604        ACS              J Org Chem   

                                       Article title  \
0  Reduced parahippocampal cortical thickness in ...   
1  Structural characterization of a Model Gram-ne...   
2  Fumaroylamino-4,5-epoxymorphinans and related ...   
3  Orvinols with mixed kappa/mu opioid receptor a...   
4  Regioselective opening of myo-inositol orthoes...   

  COST (£) charged to Wellcome (inc VAT when charged)  
0                                              £0.00   
1                        

In [52]:
colnames = []

for item in df.columns:
    firstword = ''
    for char in item:
            if char.isalnum():
                firstword += char
            else:
                break
    colnames.append(firstword.lower())

['pmid', 'publisher', 'journal', 'article', 'cost']


In [53]:
df.columns = colnames

print(df.head(5))

                    pmid publisher                 journal  \
0                    NaN       CUP  Psychological Medicine   
1             PMC3679557       ACS       Biomacromolecules   
2  23043264  PMC3506128        ACS              J Med Chem   
3    23438330 PMC3646402       ACS              J Med Chem   
4   23438216 PMC3601604        ACS              J Org Chem   

                                             article      cost  
0  Reduced parahippocampal cortical thickness in ...     £0.00  
1  Structural characterization of a Model Gram-ne...  £2381.04  
2  Fumaroylamino-4,5-epoxymorphinans and related ...   £642.56  
3  Orvinols with mixed kappa/mu opioid receptor a...   £669.64  
4  Regioselective opening of myo-inositol orthoes...   £685.88  


# 1. determine the five most common journals and the total articles for each.

In [114]:
dirty_counts = df['journal'].value_counts()
print(dirty_counts)

plos one                                                                 190
journal of biological chemistry                                           53
neuroimage                                                                29
plos pathogens                                                            24
plos genetics                                                             24
nucleic acids research                                                    23
proceedings of the national academy of sciences                           20
plos neglected tropical diseases                                          20
nature communications                                                     19
human molecular genetics                                                  19
brain                                                                     14
bmc public health                                                         14
movement disorders                                                        13

In [40]:
print(dirty_counts.keys())

Index(['PLoS One', 'PLoS ONE', 'Journal of Biological Chemistry',
       'Nucleic Acids Research',
       'Proceedings of the National Academy of Sciences',
       'Human Molecular Genetics', 'PLoS Neglected Tropical Diseases',
       'Nature Communications', 'Neuroimage', 'PLoS Pathogens',
       ...
       'Frontiers in Systems Neuroscience', 'Bioscience Reports',
       'Acta Neuropathol', 'Biologicial Chemistry', 'CHEST', 'Math Biosci',
       'Journal of Inherited Metabolic', 'Veterinary Research',
       'International Psychogeriatrics', 'Calcified Tissue International'],
      dtype='object', length=984)


In [116]:
#this cell creates 2 lists, and copies the names of each journal and the number of articles for each one
#each journal name and the number of articles for that journal have the same index in each list 
#i.e. journalnames[i] and counts[i] contain the associated values
journalnames = []
counts = []

x = 0
while x < len(dirty_counts):
    if dirty_counts.keys()[x].lower() not in journalnames:#if the journal name is not in the empty list 
        journalnames.append(dirty_counts.keys()[x].lower())#add the journal name
        counts.append(dirty_counts[x])#and add the associated count
    else:#if the journal name is already in the list
        counts[journalnames.index(dirty_counts.keys()[x].lower())] += dirty_counts[x]#add the number of articles to the existing number 
    x+=1    
print(journalnames[:5])
print(counts[:5])

['plos one', 'journal of biological chemistry', 'neuroimage', 'plos pathogens', 'plos genetics']
[190, 53, 29, 24, 24]


In [104]:
def find_n_max(names, counts, n):
        maxnames = []
        maxcounts = []
        x = 0
        while x < n:
            maxindex = counts.index(max(counts))
            maxnames.append(names.pop(maxindex))
            maxcounts.append(counts.pop(maxindex))
            x+=1
        return pd.Series(data=maxcounts, index=maxnames)

In [105]:
print(find_n_max(journalnames, counts, 5))

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
dtype: int64


# Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

Cost per article of each journal = create a dataset made up of datapoints = total costs/total number of articles for each journal, then calculate central tendency for that dataset

In [106]:
print(df.head(5))

                    pmid publisher                 journal  \
0                    NaN       CUP  psychological medicine   
1             PMC3679557       ACS       biomacromolecules   
2  23043264  PMC3506128        ACS              j med chem   
3    23438330 PMC3646402       ACS              j med chem   
4   23438216 PMC3601604        ACS              j org chem   

                                             article     cost  
0  Reduced parahippocampal cortical thickness in ...     0.00  
1  Structural characterization of a Model Gram-ne...  2381.04  
2  Fumaroylamino-4,5-epoxymorphinans and related ...   642.56  
3  Orvinols with mixed kappa/mu opioid receptor a...   669.64  
4  Regioselective opening of myo-inositol orthoes...   685.88  


In [107]:
#strip the cost strings and remove the currency symbols at the beginning and at the end
x=0
while x < len(df.index):
    df.cost[x] = df.cost[x].strip()#just in case theres any stray white space
    if not df.cost[x][0].isdigit():
        df.cost[x] = df.cost[x][1:]
    elif not df.cost[x][-1].isdigit():
        df.cost[x] = df.cost[x][:-1]
    df.cost[x] = df.cost[x].strip()
    x+=1

In [108]:
def lowercase(x):
    return str(x).lower()
df.journal = df.journal.apply(lowercase)

Cost per article of each journal = create a dataset made up of datapoints = total costs/total number of articles for each journal, then calculate central tendency for that dataset

We already have the total number of articles for each journal in the "journalnames" and "counts" arrays from the previous question. So we just need the total costs for each journal, then we can divide each one. Let's just repeat the process from the previous question but with the costs and create another array that's also lined up with journalnames and counts

In [117]:
#this cell creates 2 lists, and copies the names of each journal and the number of articles for each one
#each journal name and the number of articles for that journal have the same index in each list 
#i.e. journalnames[i] and counts[i] contain the associated values
journalnames = []
counts = []

x = 0
while x < len(dirty_counts):
    if dirty_counts.keys()[x].lower() not in journalnames:#if the journal name is not in the empty list 
        journalnames.append(dirty_counts.keys()[x].lower())#add the journal name
        counts.append(dirty_counts[x])#and add the associated count
    else:#if the journal name is already in the list
        counts[journalnames.index(dirty_counts.keys()[x].lower())] += dirty_counts[x]#add the number of articles
    x+=1    
print(journalnames[:5])
print(counts[:5])

#I ended up having to rerun this cell because I realized my implementation for the previous question POPS the journal names
#out of the list...

['plos one', 'journal of biological chemistry', 'neuroimage', 'plos pathogens', 'plos genetics']
[190, 53, 29, 24, 24]


In [121]:
totalcosts = []
#we need to fill up costs with values so that we can replace the correct indexes, we can't just append each cost
#the journals might not be in the same order as last time we made arrays like this
x=0
while x < len(journalnames):
    totalcosts.append(0)
    x+=1
#we already have all of the journal names listed in journalnames, so no need to make an array of journal names again
#just append the values in the right places
x = 0
while x < len(df.index):
    ind = journalnames.index(df.journal[x])
    totalcosts[ind] += float(df.cost[x])
    x+=1    
print(totalcosts[:5])

[7362745.359999999, 1074025.5999999999, 64239.880000000005, 2028601.0700000003, 2036146.44]


In [123]:
costperarticle = []
#journalnames, names of journals
#counts, number of articles
#totalcosts, sum of article costs

x=0
while x < len(journalnames):
    costperarticle.append(totalcosts[x]/counts[x])
    x+=1
print(costperarticle[:5])

[38751.29136842105, 20264.63396226415, 2215.168275862069, 84525.04458333335, 84839.435]


In [126]:
print('Avg cost per article: ' + str(np.mean(costperarticle)))
print('Median cost per article: ' +  str(np.median(costperarticle)))
print('Standard Deviation of the costs per article: ' + str(np.std(costperarticle)))

Avg cost per article: 21096.350030659018
Median cost per article: 1982.35
Standard Deviation of the costs per article: 120696.93704291758
